In [ ]:
!(@isdefined cleanup_hook) && IJulia.push_postexecute_hook(() -> (empty!(In); empty!(Out); GC.gc(true))); cleanup_hook = true;

In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")
include("../../../functions/utils.jl")

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure9/clustering"
mkpath(fig_path)

In [ ]:
review_fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/review_figures"
mkpath(review_fig_path)

In [ ]:
analyzer = "chuyu"

In [ ]:
datasets_corner_cue = 
[
    ["20220407_152537", 4, "jen"],
    ["20220406_111526", 9, "jen"],
    ["20220407_090156", 5, "jen"],
    ["20220417_165530", 2, "jen"],
    ["20220406_153842", 9, "jen"],
    ["20220405_171444", 25, "jen"],
    ["20220416_160516", 6, "jen"]
];

In [ ]:
@pyimport sklearn.metrics as metrics

# 2D merged neuron, PF correlation

In [ ]:
anatomy_pairwise_distance_all = []
corr_map_pairwise_all_1_all = []



data_info_all = datasets_corner_cue
for which_data = 1:length(data_info_all)
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]


    experimenter = data_info[end]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")


    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global Z_all = HDF5.readmmap(NMF_file["Z_all"])
    global X_all = HDF5.readmmap(NMF_file["X_all"])
    global Y_all = HDF5.readmmap(NMF_file["Y_all"])
    global A_dFF_1 = HDF5.readmmap(NMF_file["A_dFF"]); # bs update
    neuron_label = HDF5.readmmap(NMF_file["neuron_label"]); # bs update
    close(NMF_file)
    
    n_neuron = length(X_all)

    file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
    all_files = readdir(file_folder_1)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    n_bins = save_file_name[end-4:end-3]
    info_filename = joinpath(file_folder_1, save_file_name)
    file = h5open(info_filename, "r")
    place_cell_index = HDF5.readmmap(file["place_cell_index"])
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    close(file)
    
    # whether individual roi belongs to a certain region
    region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
    region_bool_file = h5open(region_bool_filename, "r")
    global region_names = read(region_bool_file, "region_names")
    global region_roi_bool = read(region_bool_file, "region_roi_bool")
    close(region_bool_file)


    # for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
    region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
    whether_tel = falses(n_neuron)
    for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
        if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
            whether_tel[which_neuron] = true
        end
    end

    tel_place_cell_index = intersect(place_cell_index, findall(whether_tel))
    
    
    which_neuron = tel_place_cell_index
    cell_locs = reduce(hcat, [X_all, Y_all])[which_neuron,:];
    anatomy_pairwise_distance =  metrics.pairwise_distances(cell_locs);    
    for i in 1:size(anatomy_pairwise_distance, 1)
        anatomy_pairwise_distance[i,i] = NaN
    end

    cell_locs_z = reduce(hcat, [Z_all])[which_neuron,:];
    anatomy_pairwise_distance_z =  metrics.pairwise_distances(cell_locs_z);    
    for i in 1:size(anatomy_pairwise_distance_z, 1)
        anatomy_pairwise_distance_z[i,i] = NaN
    end

    anatomy_pairwise_distance[anatomy_pairwise_distance_z.>0] .= NaN32
    
    which_neuron = tel_place_cell_index
    corr_map_pairwise_all_1 = fill(NaN32, length(which_neuron), length(which_neuron))

    @showprogress for (i, neuron_idx_1) in enumerate(which_neuron)
        for (j, neuron_idx_2) in enumerate(which_neuron)
            map_1 = place_map_all_1[:,:, neuron_idx_1]
            map_2 = place_map_all_1[:,:, neuron_idx_2]
            corr_map_pairwise_all_1[i,j] = corr_2d(map_1, map_2)
        end
    end
        
    corr_map_pairwise_all_1_copy = copy(corr_map_pairwise_all_1)
    for i in 1:size(corr_map_pairwise_all_1_copy, 1)
        corr_map_pairwise_all_1_copy[i,i] = NaN
    end

    figure()
    scatter(reshape(anatomy_pairwise_distance, length(anatomy_pairwise_distance)), reshape(corr_map_pairwise_all_1_copy, length(corr_map_pairwise_all_1_copy)), s= 0.005)
    xlabel("anatomy_pairwise_distance")
    ylabel("corr_map_pairwise_all_1") 
        
    

    h5open(joinpath(data_path(ds_save_analyzer_1), "anatomy_PF_correlation_2d.h5"), "w") do file
        file["corr_map_pairwise_all_1"] = corr_map_pairwise_all_1
        file["tel_place_cell_index"] = tel_place_cell_index
        file["anatomy_pairwise_distance"] = anatomy_pairwise_distance  
        file["anatomy_pairwise_distance_z"] = anatomy_pairwise_distance_z
    end;

        
    append!(anatomy_pairwise_distance_all, [reshape(anatomy_pairwise_distance, length(anatomy_pairwise_distance))])
    append!(corr_map_pairwise_all_1_all, [reshape(corr_map_pairwise_all_1_copy, length(corr_map_pairwise_all_1_copy))])
    
end

In [ ]:
data_info_all = datasets_corner_cue
for which_data = 1:length(data_info_all)
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]


    experimenter = data_info[end]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")


    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global Z_all = HDF5.readmmap(NMF_file["Z_all"])
    global X_all = HDF5.readmmap(NMF_file["X_all"])
    global Y_all = HDF5.readmmap(NMF_file["Y_all"])
    global A_dFF_1 = HDF5.readmmap(NMF_file["A_dFF"]); # bs update
    neuron_label = HDF5.readmmap(NMF_file["neuron_label"]); # bs update
    close(NMF_file)
    
    n_neuron = length(X_all)

    file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
    all_files = readdir(file_folder_1)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    n_bins = save_file_name[end-4:end-3]
    info_filename = joinpath(file_folder_1, save_file_name)
    file = h5open(info_filename, "r")
    place_cell_index = HDF5.readmmap(file["place_cell_index"])
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    close(file)
    
    # whether individual roi belongs to a certain region
    region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
    region_bool_file = h5open(region_bool_filename, "r")
    global region_names = read(region_bool_file, "region_names")
    global region_roi_bool = read(region_bool_file, "region_roi_bool")
    close(region_bool_file)


    # for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
    region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
    whether_tel = falses(n_neuron)
    for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
        if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
            whether_tel[which_neuron] = true
        end
    end

    tel_place_cell_index = intersect(place_cell_index, findall(whether_tel))
    
    
    which_neuron = tel_place_cell_index
    cell_locs = reduce(hcat, [X_all, Y_all])[which_neuron,:];
    anatomy_pairwise_distance =  metrics.pairwise_distances(cell_locs);    
    for i in 1:size(anatomy_pairwise_distance, 1)
        anatomy_pairwise_distance[i,i] = NaN
    end

    cell_locs_z = reduce(hcat, [round.(Z_all)])[which_neuron,:];
    anatomy_pairwise_distance_z =  metrics.pairwise_distances(cell_locs_z);    
    for i in 1:size(anatomy_pairwise_distance_z, 1)
        anatomy_pairwise_distance_z[i,i] = NaN
    end

    anatomy_pairwise_distance[anatomy_pairwise_distance_z.>0] .= NaN32
    


    h5open(joinpath(data_path(ds_save_analyzer_1), "anatomy_PF_correlation_2d.h5"), "r+") do file
        if haskey(file, "anatomy_pairwise_distance")
            delete_object(file, "anatomy_pairwise_distance")
        end

        if haskey(file, "anatomy_pairwise_distance_z")
            delete_object(file, "anatomy_pairwise_distance_z")
        end
        file["anatomy_pairwise_distance"] = anatomy_pairwise_distance
        file["anatomy_pairwise_distance_z"] = anatomy_pairwise_distance_z
    end;

    
end

# Check results

In [ ]:
anatomy_pairwise_distance_all = []
corr_map_pairwise_all_1_all = []
anatomy_pairwise_distance_z_all = []

exclude_percent = []

data_info_all = datasets_corner_cue
for which_data = 1:length(data_info_all)
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]


    experimenter = data_info[end]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    # whether individual roi belongs to a certain region
    anatomy_PF_correlation_filename = joinpath(data_path(ds_save_cy_1), "anatomy_PF_correlation_2d.h5")
    anatomy_PF_correlation_file = h5open(anatomy_PF_correlation_filename, "r")
    corr_map_pairwise_all_1 = read(anatomy_PF_correlation_file, "corr_map_pairwise_all_1")
    anatomy_pairwise_distance = read(anatomy_PF_correlation_file, "anatomy_pairwise_distance")
    anatomy_pairwise_distance_z = read(anatomy_PF_correlation_file, "anatomy_pairwise_distance_z")
    mask_overlap = read(anatomy_PF_correlation_file, "mask_overlap")
    close(anatomy_PF_correlation_file)

        
    
    anatomy_pairwise_distance_copy = copy(anatomy_pairwise_distance)
    for i in 1:size(anatomy_pairwise_distance_copy, 1)
        anatomy_pairwise_distance_copy[i,i] = NaN
    end

        
    corr_map_pairwise_all_1_copy = copy(corr_map_pairwise_all_1)
    for i in 1:size(corr_map_pairwise_all_1_copy, 1)
        corr_map_pairwise_all_1_copy[i,i] = NaN
    end      

    corr_map_pairwise_all_1_copy[isnan.(anatomy_pairwise_distance_copy)] .= NaN

    nr_pair = length(findall(isfinite.(corr_map_pairwise_all_1_copy)))

    corr_map_pairwise_all_1_copy[mask_overlap] .= NaN32

    nr_pair_masked = length(findall(isfinite.(corr_map_pairwise_all_1_copy)))

    append!(exclude_percent, (1 - nr_pair_masked/nr_pair)*100)

        
    append!(anatomy_pairwise_distance_all, [reshape(anatomy_pairwise_distance_copy, length(anatomy_pairwise_distance_copy))])
    append!(corr_map_pairwise_all_1_all, [reshape(corr_map_pairwise_all_1_copy, length(corr_map_pairwise_all_1_copy))])
    append!(anatomy_pairwise_distance_z_all, [reshape(anatomy_pairwise_distance_z, length(anatomy_pairwise_distance_z))])
    
end

In [ ]:
mean(exclude_percent)

In [ ]:
std(exclude_percent)

In [ ]:
x = reduce(vcat, anatomy_pairwise_distance_all)
y = reduce(vcat, corr_map_pairwise_all_1_all)

bins = numpy.arange(5,nanmaximum(x)+2.5,5)

regress_x, regress_y, y_std, bin_numbers = py"smooth_scatter_sem"(x[isfinite.(x).*isfinite.(y)],y[isfinite.(x).*isfinite.(y)],bins =bins)


fig,ax = subplots(1,1,figsize=(1.2,1), dpi=250)

ax.plot(regress_x, regress_y, color="r")
ax.fill_between(regress_x, regress_y-y_std, regress_y+y_std, alpha=0.3, color="r")


xlabel("Anatomical distance (μm)")
ylabel("PF correlation")


ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)
xticks([0,100,200])



ylim(-1,1)

# axvline(20, color="k",linestyle="dashed")
# ax.text(30,0.3, "d = 20 μm")

fig.savefig(joinpath(review_fig_path, "distance_anatomy_PF_correlation.pdf"), bbox_inches="tight",transparent = true);